In [1]:
using StatsBase
using Distributions
using PyPlot

col, bgcol, grdcol, lty, ticksize = ("black", "white", "#848482", "dotted", 8)

("black","white","#848482","dotted",8)

In [3]:
function xyplot(x::AbstractArray{Float64, 1}, y::AbstractArray{Float64, 1}; xlab::AbstractString = "", ylab::AbstractString  = "",
  xlab_fontsize::Int = 15, ylab_fontsize::Int = 15,
  color::AbstractString = "", bgcol::AbstractString = "#FFFFFF",
  marker::AbstractString = "o", markersize::Int = 5, grid_lty::AbstractString = "solid",
  linestyle::AbstractString = "None", linewidth::Float64 = 1.5, ticksize::Int = 12, grid_color::AbstractString = "#F2F3F4",
  figsize::Tuple = (6, 6), file_name::AbstractString = "", save::Bool = true, add::Bool = false, panel::Function = function fun() end)
    matplotlib[:rc]("font", family = "Arial")
    matplotlib[:rc]("xtick", direction = "out")
    matplotlib[:rc]("ytick", direction = "out")

    """ X-Y PLOT
    --------------------------
    x - input for x-axis
    y - input for y-axis
    """

    if add == true
      plt[:gcf]()
      plt[:gca]()
      if color == ""
        plt[:plot](x, y, marker = marker, markersize = markersize, linestyle = linestyle, linewidth = linewidth)
      else
        plt[:plot](x, y, color = color, marker = marker, markersize = markersize, linestyle = linestyle, linewidth = linewidth)
      end
    else
      plt[:cla](); plt[:clf]()
      f, ax = subplots(figsize = figsize)
      if color == ""
        plt[:plot](x, y, marker = marker, markersize = markersize, linestyle = linestyle, linewidth = linewidth)
      else
        plt[:plot](x, y, color = color, marker = marker, markersize = markersize, linestyle = linestyle, linewidth = linewidth)
      end
      plt[:minorticks_on]()
      ax[:set_xlabel](xlab, fontsize = xlab_fontsize)
      ax[:set_ylabel](ylab, fontsize = ylab_fontsize)
      ax[:grid]("on", which = "major", color = grid_color, linestyle = grid_lty)
      ax[:set_axis_bgcolor](bgcol)
      ax[:tick_params](axis = "both", which = "major", labelsize = ticksize, pad = 5)
      ax[:tick_params](axis = "both", which = "minor", labelsize = ticksize, pad = 5)
      ax[:set_axisbelow]("on")
      ax[:locator_params](axis = "x", nbins = 7, tight = true)
      ax[:locator_params](axis = "y", nbins = 7, tight = true)
      ax[:margins](.03)
    end

    if save == true
      if file_name == ""
        savefig(joinpath(pwd(), "Current Plot.png"), bbox_inches = "tight", dpi = 200)
        plt[:cla]()
        plt[:clf]()
        plt[:close]()
      else
        savefig(file_name, bbox_inches = "tight", dpi = 200)
        plt[:cla]()
        plt[:clf]()
        plt[:close]()
      end
    else
      plt[:show]()
    end
end


xyplot (generic function with 1 method)

In [4]:
using PyPlot
using StatsBase: fit, Histogram

nsample = 80000;
xStep = 0.01;
m = 1; # dK
C = 3;
dt = 0.1;
nstep = 50;
V = 4;


srand(1234);

r = 80000;
xStep = 0.01;
ɛ = .1;
τ = 50;

# set up functions
U(x) = -2(x.^2) + x.^4;
dU(x) = -4x + 4(x.^3);
dU2(x) = -4x + 4(x.^3) + 2*rand(Normal());
K(p) = (p * p) / 2;
dK(p) = p;
H(x, p) = U(x) + K(p);


#gradUPerfect(x) = -4x + 4(x.^3);
"""
HMC without noise with M-H
"""
x1 = Array{Float64, 1}(); push!(x1, 0)

for n in 1:(r - 1)
  xNew = x1[n]
  p = rand(Normal())
  oldE = H(xNew, p)

  for τ_i in 1:τ
    p = p - (ɛ / 2) * dU(xNew)
    xNew = xNew + ɛ * dK(p)
    p = p - (ɛ / 2) * dU(xNew)
  end

  newE = H(xNew, p)
  dE = newE - oldE

  if dE < 0
    push!(x1, xNew)
  elseif rand(Uniform()) < exp(-dE)
    push!(x1, xNew)
  else
    push!(x1, x1[n])
  end

end

In [5]:
"""
HMC without noise with no M-H
"""
x2 = Array{Float64, 1}(); push!(x2, 0)

for n in 1:(r - 1)
  xNew = x2[n]
  p = rand(Normal())
  oldE = H(xNew, p)

  for τ_i in 1:τ
    p = p - (ɛ / 2) * dU(xNew)
    xNew = xNew + ɛ * dK(p)
    p = p - (ɛ / 2) * dU(xNew)
  end

  push!(x2, xNew)
end

In [6]:
"""
HMC with noise with M-H
"""
x3 = Array{Float64, 1}(); push!(x3, 0)

for n in 1:(r - 1)
  xNew = x3[n]
  p = rand(Normal())
  oldE = H(xNew, p)

  for τ_i in 1:τ
    p = p - (ɛ / 2) * dU2(xNew)
    xNew = xNew + ɛ * dK(p)
    p = p - (ɛ / 2) * dU2(xNew)
  end

  newE = H(xNew, p)
  dE = newE - oldE

  if dE < 0
    push!(x3, xNew)
  elseif rand(Uniform()) < exp(-dE)
    push!(x3, xNew)
  else
    push!(x3, x3[n])
  end

end


In [7]:
"""
HMC without noise with no M-H
"""
x4 = Array{Float64, 1}(); push!(x4, 0)

for n in 1:(r - 1)
  xNew = x4[n]
  p = rand(Normal())
  oldE = H(xNew, p)

  for τ_i in 1:τ
    p = p - (ɛ / 2) * dU2(xNew)
    xNew = xNew + ɛ * dK(p)
    p = p - (ɛ / 2) * dU2(xNew)
  end

  push!(x4, xNew)
end


In [13]:
"""
Stochastic Gradient HMC
"""
x5 = Array{Float64, 1}(); push!(x5, 0)
for n in 1:(r - 1)
  p = rand(Normal()) * sqrt(1);
  B = 0.5 * V * ɛ;
  D = sqrt(2 * (C - B) * ɛ);

  for i = 1:τ
      p = p - dU2(xNew) * ɛ  - p * C * ɛ  + rand(Normal()) * D;
      x5[n + 1] = x5[n] + dK(p) * dt;
  end

  push!(x5, xNew)
end



LoadError: LoadError: BoundsError: attempt to access 1-element Array{Float64,1} at index [2]
while loading In[13], in expression starting on line 5

In [ ]:
"""
Plot the Target Distribution and HMCs
"""
xGrid = collect(-3:xStep:3);
y = exp(-U(xGrid));
y = y / sum(y) / xStep;
xhmc1, yhmc1 = hist(x, xGrid);
yhmc1 = yhmc1 / sum(yhmc1) / xStep;

xhmc2, yhmc2 = hist(x2, xGrid);
yhmc2 = yhmc2 / sum(yhmc2) / xStep;

xhmc3, yhmc3 = hist(x3, xGrid);
yhmc3 = yhmc3 / sum(yhmc3) / xStep;

xhmc4, yhmc4 = hist(x4, xGrid);
yhmc4 = yhmc4 / sum(yhmc4) / xStep;

xhmc5, yhmc5 = hist(x5, xGrid);
yhmc5 = yhmc5 / sum(yhmc5) / xStep;
xyplot(xGrid, y, linestyle = "-", marker = "", save = false);
xyplot(xhmc1[2:end], yhmc1, linestyle = "-", marker = "", add = true, save = false);
xyplot(xhmc2[2:end], yhmc2, linestyle = "-", marker = "", add = true, save = false);
xyplot(xhmc3[2:end], yhmc3, linestyle = "-", marker = "", add = true, save = false);
xyplot(xhmc4[2:end], yhmc4, linestyle = "-", marker = "", add = true, save = false);
xyplot(xhmc5[2:end], yhmc5, linestyle = "-", marker = "", add = true);

"""
Plot the Autocorrelation
"""
barplot(autocor(x1))
barplot(autocor(x2))
barplot(autocor(x3))
barplot(autocor(x4))
barplot(autocor(x5))